In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf
import pyarrow

In [2]:
df_ama = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\amazon_prime_titles.csv', sep = ',')
df_dis = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\disney_plus_titles.csv', sep = ',')
df_hul = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\hulu_titles.csv', sep = ',')
df_net = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\netflix_titles.csv', sep = ',')

In [3]:
df_1 = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\ratings\1.csv', sep = ',')
df_2 = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\ratings\2.csv', sep = ',')
df_3 = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\ratings\3.csv', sep = ',')
df_4 = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\ratings\4.csv', sep = ',')
df_5 = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\ratings\5.csv', sep = ',')
df_6 = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\ratings\6.csv', sep = ',')
df_7 = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\ratings\7.csv', sep = ',')
df_8 = pd.read_csv(r'C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\ratings\8.csv', sep = ',')

In [4]:
df_viwers=pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8])

In [5]:
df_viwers = df_viwers.rename(columns={'rating':'score'})
df_viwers = df_viwers.rename(columns={'movieId':'id'})

In [6]:
df_viwers['timestamp'] = pd.to_datetime(df_viwers['timestamp'], unit='s', origin='unix')
df_viwers['timestamp'] = pd.to_datetime(df_viwers['timestamp']).dt.strftime('%Y-%m-%d')


In [7]:
df_ama["source"] = "amazon"
df_dis["source"] = "disney"
df_hul["source"] = "hulu"
df_net["source"] = "netflix"

In [8]:
df_movies = pd.concat([df_ama,df_dis,df_hul,df_net])

In [9]:
df_movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,source
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,amazon
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,amazon
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,amazon
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",amazon
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,amazon


In [10]:
df_movies["id"] = df_movies["source"].str[0] + df_movies["show_id"]


In [11]:
df_movies["rating"].fillna("G", inplace=True)


In [12]:
df_movies['date_added'] = pd.to_datetime(df_movies['date_added'])
df_movies['date_added'] = df_movies['date_added'].dt.strftime('%Y-%m-%d')


In [13]:
df_movies = df_movies.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [14]:
df_completo = pd.merge(df_viwers, df_movies, on='id', how='inner')


In [15]:

promedios = df_viwers.groupby("id")["score"].mean()
promedios_redondeados = promedios.round(2)
score_medio = pd.DataFrame({'id': promedios_redondeados.index, 'score_medio': promedios_redondeados.values})
score_medio

,id,score_medio
0,as1,3.47
1,as10,3.44
2,as100,3.61
3,as1000,3.56
4,as1001,3.59
...,...,...
22993,ns995,3.52
22994,ns996,3.63
22995,ns997,3.53
22996,ns998,3.58


In [16]:
df_consultas = pd.merge(df_movies,score_medio, on='id', how='inner')

In [17]:
df_completo.to_parquet('df_completo.parquet')